In [1]:
import pandas as pd
sheets=pd.read_excel('content.xlsx',None)
#df_all=pd.concat(sheets)

In [2]:
import nltk
#nltk.download() #download nltk once
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
from gensim.summarization.summarizer import summarize as gsum
from gensim.summarization import keywords
from summa.summarizer import summarize as ssum

import spacy
data={'doc_id':[],'sheet_id':[],'text':[]}
for s in sheets.keys():
    for k,v in sheets[s].iterrows():
        try:
            if len(v['text'])>10:
                d=v['doc_id']
                a=v['text']
                data['doc_id'].append(d)   
                data['sheet_id'].append(s)
                data['text'].append(a)

    #             sentences=[]
    #             for s in sent_tokenize(a):
    #                 names=[]
    #                 for e in nlp(s).ents if e.label_ not in exclude:
    #                     names.appendend((e.text,e.label_))
    #                 sentences.append(s,names)
            else:
                print(d,'abstract too short',a)
        except:
            print(v['doc_id'],'unable to read abstract')
df=pd.DataFrame(data)

In [3]:
#nltk.download()

In [4]:
from symspellpy.symspellpy import SymSpell, Verbosity

encoding="utf-8"
corpus_file="corpus.txt"
dict_file="frequency_dictionary.txt"
dict_min_freq=2

bigram_file="frequency_bigram.txt"
bigram_min_freq=2

max_edit_distance_dictionary = 2
max_edit_distance_lookup = 2
prefix_length = 7

output_json='corpus.json'

with open(corpus_file, "w+", encoding=encoding) as f:
    for k,(p,sh,a) in df.iterrows():
        #print("{}".format(a))
        #print(a)
        f.write("{}\n\n".format(a))

In [5]:
#dict
#http://www.javascriptthai.com/wp-content/uploads/2013/11/Windows-874-Unicode-1.gif
import os
import re
from nltk.collocations import BigramCollocationFinder
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
dict_fd={}
bigram_fd={}
def parse_words(text):
    matches = re.findall(r"(([^\W_]|['’-])+)", text.lower())
    #matches+= re.findall(r"(?::?%[nmp])", text)
    #matches+= re.findall(r"(^[u0E01-u0E5B]+$/)", text)
    #(?::?%[nmp])
    #[u0E01-u0E5B]
    #[^\u0E00-\u0E7F]
    #[^\u0E00-\u0E7Fa-zA-Z' ]
    matches = [match[0] for match in matches]
    #print(matches)
    return matches

def sort_dict_by_value(x, reverse=True):
    return {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=reverse)}

def create_dict_from_sent(text,min_freq=1):
    #tokens = word_tokenize(text)
    tokens=parse_words(text)
    for w in tokens:
        dict_fd[w]=dict_fd.get(w,0)+1
        
def create_dict(corpus_file, dict_file="frequency_dict.txt", encoding=encoding, min_freq=1):
    if not os.path.exists(corpus_file):
        return False
    with open(corpus_file, "r", encoding=encoding) as infile:
        for para in infile:
            sents=sent_tokenize(para)
            for text in sents:
                create_dict_from_sent(text)
    with open(dict_file, "w+", encoding=encoding) as f:
        for word,count in sort_dict_by_value( dict_fd, reverse=True ).items():
            if count>=min_freq:
                #print("{} {}".format(word,count))
                f.write("{} {}\n".format(word, count))
                
def create_bigram_from_sent(text,window_size=2,min_freq=1):
    #tokens = word_tokenize(text)
    tokens=parse_words(text)
    finder = BigramCollocationFinder.from_words(tokens,window_size=window_size)
    finder.apply_freq_filter(min_freq=min_freq)
    for pair,count in sort_dict_by_value( finder.ngram_fd, reverse=True).items():
        #print(pair,count)
        bigram_fd[pair]=bigram_fd.get(pair,0)+1
        
def create_bigram(corpus_file, bigram_file="frequency_bigram.txt", encoding=encoding,min_freq=1,window_size=2):
    if not os.path.exists(corpus_file):
        return False
    with open(corpus_file, "r", encoding=encoding) as infile:
        for para in infile:
            sents=sent_tokenize(para)
            for text in sents:
                create_bigram_from_sent(text,window_size=window_size)
    with open(bigram_file, "w+", encoding=encoding) as f:
        for pair, count in sort_dict_by_value( dict(bigram_fd), reverse=True ).items():
            if count>=min_freq:
                #print("{} {}".format(pair, count))
                f.write("{} {} {}\n".format(pair[0],pair[1], count))



In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
create_dict(corpus_file=corpus_file, dict_file=dict_file, min_freq=dict_min_freq);
create_bigram(corpus_file=corpus_file, bigram_file=bigram_file, min_freq=bigram_min_freq);

In [8]:
from collections import Counter
from tqdm.notebook import tqdm
nlp = spacy.load("en_core_web_sm")
exclude=['PERCENT','ORDINAL','CARDINAL']

def get_keyword(text):
    sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)
    if not sym_spell.load_dictionary(dict_file, term_index=0, count_index=1):
        print("Dictionary file not found")
    if not sym_spell.load_bigram_dictionary(bigram_file, term_index=0, count_index=2):
        print("Bigram dictionary file not found")    
    suggestions=sym_spell.lookup_compound(text, max_edit_distance_lookup)
    kw=keywords(suggestions[0].term, words=20, split=True, scores=False, lemmatize=True)
    return kw

#     'sentence': [
#         (s, [ (e.text,e.label_) for e in nlp(s).ents if e.label_ not in exclude ])
#         for s in sent_tokenize(a)\
#     ],
def extract(df):
    return [{
            'doc_id': p,
            'eng_abstract': a,
            'keywords': get_keyword(a),
            'name': sort_dict_by_value(Counter([e.text for e in nlp(a).ents if e.label_ not in exclude])),
            'summarization':ssum(a),
        } for k,(p,sh,a) in df.iterrows()]

%%timeit -n1
df2=df[:8]
import multiprocessing
n=multiprocessing.cpu_count()
pool = multiprocessing.Pool()
list(pool.map( extract,[df2[df2.index%i==0] for i in range(n)]))

In [9]:
df

,doc_id,sheet_id,text
0,data/sport/287.txt,sport,France coach Bernard Laporte has made four cha...
1,data/sport/253.txt,sport,Celtic's Henri Camara and Nacho Novo of Ranger...
2,data/sport/012.txt,sport,World outdoor triple jump record holder and BB...
3,data/sport/232.txt,sport,Glenn Hoddle has been unveiled as the new Wolv...
4,data/sport/202.txt,sport,Tottenham manager Jacques Santini has resigned...
...,...,...,...
2219,data/tech/091.txt,tech,The body that represents the US movie industry...
2220,data/tech/200.txt,tech,Your computer can now help solve the world's m...
2221,data/tech/173.txt,tech,The US is poised to push Japan off the top of ...
2222,data/tech/393.txt,tech,Internet search engine users are an odd mix of...


In [ ]:
df2=df
from functools import reduce
import multiprocessing
n=multiprocessing.cpu_count()
pool = multiprocessing.Pool()
content=reduce(lambda x,y:x+y, pool.map( extract,[df2[df2.index%i==0] for i in range(n)]))

In [ ]:
import json
with open(output_json, 'w+',encoding=encoding) as fp:
    json.dump(content, fp)

In [ ]:
with open(output_json, 'r', encoding=encoding) as fp:
    content2 = json.load(fp)

In [ ]:
content

In [ ]:
len(content2)